In [1]:
import json
import requests
import time
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
dt = 'apple'
with open(f'/gdrive/MyDrive/Data/{dt}/questions_related.json') as j :
  questions = pd.read_json(j,orient='table') 

In [20]:
tokens = {1:{'access_token': 'grmO1cXoVtH4cQKNGKQoQw))','key':'NgwXtZS5zddMDTN5ch8kUQ(('},
          2:{'access_token': 'pe*iEaXctSGrih1mUvrIbA))','key':'t5sJBRionMBtc05aetiCmw(('},
          3:{'access_token': '4y4d4geC36lSDcJTpqx1aQ))','key':'j3YCFivyCrWDhDH*ltdokQ(('},
          4:{'access_token': 'eJS8fzitX3qDl2AnJY*P8g))','key': 'Cp1AgenjkXQ03I7avs*jDg(('},
          5:{'access_token': 'hEP9Sl6CZDgx9RzSx9BLmw))','key':'b6*O3b8)i5eVM4CdcoQ6IA(('},
          6:{'access_token':'cUegVFv0nnjCDJk545(mMw))','key':'2A2uI0hRs6rhOr4n5spMGQ(('}
          }
token_counter = 1

In [6]:
print_full(questions.head())

        AcceptedAnswerId          AnswerCount                         AnswerIds                           Attachments     OS  PostTypeId                                                                  Related  Score                                        Tags                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [7]:
def limit_checker(i):
    url = f"https://api.stackexchange.com/2.2/questions/28/related?access_token={tokens[i]['access_token']}&key={tokens[i]['key']}&order=desc&sort=activity&site={dt}&filter=!--Fgz)TFKJfq"
    a = requests.get(url)
    print(a)
    related = json.loads(a.content)
    
    # print(related)
    return related['quota_remaining']

In [8]:
limit = limit_checker(token_counter)
print(limit)

<Response [200]>
9999


#Init

Trial, Pandas dataframe creation 🤢 

In [9]:
url = f"https://api.stackexchange.com/2.2/questions/101090/linked?access_token={tokens[token_counter]['access_token']}&key={tokens[token_counter]['key']}&order=desc&sort=activity&site=unix&filter=!--MdnFsTG2ty"
a = requests.get(url)

In [10]:
linked = json.loads(a.content)

In [11]:
print(linked)

{'items': [], 'has_more': False, 'quota_max': 10000, 'quota_remaining': 9999}


In [ ]:
# print(linked['items'][0]['question_id'] in questions.index.values)

In [13]:
a = pd.Series(data = list(questions.loc[28])+[[]], index = list(questions.columns) + ['Linked'] )
linked_pd = pd.DataFrame([a])
linked_pd.at[0,'Id'] = 28
linked_pd.set_index('Id',inplace = True)

In [14]:
print_full(linked_pd)

          AcceptedAnswerId          AnswerCount   AnswerIds        Attachments   OS  PostTypeId                                                              Related  Score               Tags                                                                                                                                                                                                                                                                                                              Text                                   Title Linked
Id                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [15]:
# linked_pd.at[29447,'Title'] = 'Abracadabra'
# print(type(linked_pd.at[29447,'Tags']))

#Coding

In [16]:
def create_new_entry(i,data):
    linked_pd.at[i,'AnswerCount'] = data['answer_count']
    linked_pd.at[i,'Score'] = data['score']
    linked_pd.at[i,'Title'] = data['title']
    linked_pd.at[i,'Text'] = data['body']
    linked_pd.at[i,'PostTypeId'] = 1
    linked_pd.at[i,'OS'] = 'unix'
    linked_pd.at[i,'Tags'] = []
    linked_pd.at[i,'Tags'] = data['tags']
    if 'accepted_answer_id' in data:
        linked_pd.at[i,'AcceptedAnswerId'] = data['accepted_answer_id']

In [21]:
print(limit_checker(token_counter))

<Response [503]>


JSONDecodeError: ignored

In [18]:
#@title Default title text
cnt = 0 
pairs = 0
limit = limit_checker(token_counter)
# trial = 10
for i in questions.index.values:
    if(i<103641):
        continue
    # if(pairs>trial):
    #     break
    if(pairs%20==0 and pairs!=0):
        print(f'{pairs} pairs has been formed')
    if(cnt>limit):
        if(token_counter==6):
            print('All the tokens has been exhausted')
            break
        print('Token no- '+str(token_counter)+'has been exhausted. No of queries served = '+str(cnt))
        token_counter+=1
        cnt=0
        limit = limit_checker(token_counter)
    start = time.time()
    url = f"https://api.stackexchange.com/2.2/questions/{i}/linked?access_token={tokens[token_counter]['access_token']}&key={tokens[token_counter]['key']}&order=desc&sort=activity&site={dt}&filter=!4(L6loqBHp1ct1eg1"
    cnt+=1
    a = requests.get(url)
    linked = json.loads(a.content)
    try:
        if(len(linked['items'])!=0):
            for j in range(0,len(linked['items'])):
                id_link = linked['items'][j]['question_id']
                if(i in list(linked_pd.index.values)):
                    if((id_link in linked_pd.at[i,'Linked'])==0):
                        linked_pd.at[i,'Linked'].append(id_link)
                        pairs+=1
                        if(id_link in list(linked_pd.index.values)):
                            linked_pd.at[id_link,'Linked'].append(i)
                        else:
                            create_new_entry(id_link,linked['items'][j])
                            linked_pd.at[id_link,'Linked'] = []
                            linked_pd.at[id_link,'Linked'].append(i)
                else:
                    linked_pd.at[i] = questions.loc[i]
                    linked_pd.at[i,'Linked'] = []
                    linked_pd.at[i,'Linked'].append(id_link)
                    pairs+=1
                    if(id_link in list(linked_pd.index.values)):
                            linked_pd.at[id_link,'Linked'].append(i)
                    else:
                        create_new_entry(id_link,linked['items'][j])
                        linked_pd.at[id_link,'Linked'] = []
                        linked_pd.at[id_link,'Linked'].append(i)
    except:
        print(f'Something went wrong with the items of id {i} ')

    end = time.time()
    if(end-start<0.04):
        time.sleep(0.04-(end-start))

<Response [200]>
40 pairs has been formed
40 pairs has been formed
40 pairs has been formed
40 pairs has been formed


ConnectionError: ignored

In [ ]:
print(pairs)
print(linked_pd.info())
print_full(linked_pd.tail())

In [ ]:
linked_pd.to_json('/gdrive/MyDrive/Data/apple/apple_linked.json',orient='table')
linked_pd.to_json('/content/apple_linked.json',orient='table')

In [56]:
!sudo apt-get install net-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 194 kB of archives.
After this operation, 803 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 net-tools amd64 1.60+git20161116.90da8a0-1ubuntu1 [194 kB]
Fetched 194 kB in 1s (361 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package net-tools.
(Reading database ... 145483 files

In [1]:
!curl ipecho.net/plain

35.237.59.19